<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports e definições de funções

In [69]:
# !pip install tensorflow
# !pip install keras

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import json
import sklearn
import numpy as np
import pandas as pd
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.layers import Dropout, Conv1D, MaxPooling1D, Flatten, Dense, SpatialDropout1D, BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, RMSprop, Adagrad, Adamax, Nadam, Ftrl
from keras.models import Sequential
from keras.layers.embeddings import Embedding

from mlxtend.plotting import plot_confusion_matrix
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

In [72]:
def tokenize(tweets_list):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweets_list)
    return tokenizer.texts_to_sequences(tweets_list), tokenizer

In [73]:
def pad(text_tokenized, length=None):
    return pad_sequences(text_tokenized, maxlen=length, padding='post')

In [74]:
def preprocess(tweets_list, max_text_length):
   
    preprocess_tweets_list, tweets_list_tokenizer = tokenize(tweets_list)

    preprocess_tweets_list = pad(preprocess_tweets_list, length=max_text_length)

    return preprocess_tweets_list, tweets_list_tokenizer

In [75]:
# def logits_to_text(logits, tokenizer):
#     index_to_words = {id: word for word, id in tokenizer.word_index.items()}
#     index_to_words[0] = '<PAD>'

#     return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [76]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            self.score = roc_auc_score(self.y_val, y_pred)
            self.false_positive_rate, self.true_positive_rate, _ = roc_curve(self.y_val, y_pred) 
                   

# Carrega arquivo de saída (resultados)

In [77]:
# CONSTANTES DOS RESULTADOS
REDE = 'CNN'

BASE_0 = 'BASE 0'
BASE_1 = 'BASE 1'
BASE_2 = 'BASE 2'

# Caminho arquivo de saída
PATH_ARQ_SAIDA = "/content/drive/MyDrive/TCC/resultados/resultados.csv"

#  rede base otimizador acuracia val_loss learning_rate dropout batch_size roc_curve train_resume confusion_matrix
resultados = pd.read_csv(PATH_ARQ_SAIDA, index_col=0)
resultados.head()

,rede,base,otimizador,acuracia,val_loss,learning_rate,dropout,batch_size,roc_curve,train_resume,confusion_matrix


# Carrega base

In [79]:
# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"
#MATRIZ_CBOW_300_BASE_1 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV", delimiter=',')

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"
#MATRIZ_CBOW_300_BASE_2_LEILA = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV", delimiter=',')

# Base0 Bruta Não processada
PATH_BASE0_NAO_PROCESSADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base0_classificada_balanceada.csv"

PATH_BASE_1_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1_classificada_balanceada.csv" #entrada

PATH_BASE_2_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv" #entrada
MATRIZ_CBOW_300_BASE_2 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_balanceada_CBOW300.csv", delimiter=',') #saida

In [99]:
tweets = pd.read_csv(PATH_BASE0_NAO_PROCESSADA, index_col=0)
text_column = tweets['text']
text_column

0       RT @ThomasM71708305: @allantercalivre A Bahia ...
1       @allantercalivre @gen_heleno @CNNBrasil O Gene...
2       @allantercalivre @gen_heleno @CNNBrasil Alan t...
3       @allantercalivre @OlavoOpressor @opropriolavo ...
4       RT @ThomasM71708305: @allantercalivre A Bahia ...
                              ...                        
9269          simplesmente amoooo https://t.co/I12jVsUdvb
9270    @Mary_Greicy Abençoado Ano Novo  sua linda e q...
9271    @MarcusConcolato leva eles com a gente, eles s...
9272    se tu NUNCA fumou maconha cole isso no seu mur...
9273           @trizbeiro Kkkkk kkk ata eu sou meia sonsa
Name: text, Length: 9274, dtype: object

In [100]:
# Pega a média de caracteres dos tweets de toda a base
max_text_length = int(text_column.apply(lambda x: len(str(x).split(' '))).max())
max_text_length

68

In [101]:
output_label = tweets['label']
input_data, text_tokenizer = preprocess(text_column, None)
    
text_vocab_size = len(text_tokenizer.word_index)
print("Vocabulary size:", text_vocab_size)

Vocabulary size: 24839


# Prepara os dados de train, test e validation da base


In [103]:
# Carrega os paths das divisões das bases
LIST_PATH_TRAIN = []
LIST_PATH_TEST = []
LIST_PATH_VAL = []
for i in range(3):
  LIST_PATH_TRAIN.append('/content/drive/MyDrive/TCC/dados/processadas/bases_balanceadas_divididas/train'+str(i)+'.csv')
  LIST_PATH_TEST.append('/content/drive/MyDrive/TCC/dados/processadas/bases_balanceadas_divididas/test'+str(i)+'.csv')
  LIST_PATH_VAL.append('/content/drive/MyDrive/TCC/dados/processadas/bases_balanceadas_divididas/val'+str(i)+'.csv')

In [104]:
# 0 - Base0 bruta não processada
# 1 - Base1 Com StopWords
# 2 - Base2 Sem StopWords
base_train = pd.read_csv(LIST_PATH_TRAIN[2], index_col=0)
base_test = pd.read_csv(LIST_PATH_TEST[2], index_col=0)
base_val = pd.read_csv(LIST_PATH_VAL[2], index_col=0)

In [105]:
# Dados train
x_train, text_train_tokenizer = preprocess(base_train['text'], max_text_length) # o max_text_length usado é o da base inteira
y_train = base_train['label']
# Dados test
x_test, text_test_tokenizer = preprocess(base_test['text'], max_text_length)
y_test = base_test['label']
# Dados validation    
x_val, text_val_tokenizer = preprocess(base_val['text'], max_text_length)
y_val = base_val['label']

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(2930, 68)
(978, 68)
(978, 68)


# Execução modelo

In [106]:
# parametros tunning:
BATCH_SIZE = [32, 64]
LEARN_RATE = [0.0001, 0.001]
OTMIZADORES = ['Nadam', 'RMSProp', 'SGD']
DROPOUT = [0.1, 0.2, 0.3]

# parametros estaticos:
EPOCHS = 10
VAL_AND_TST_SIZE = 0.2 # Fazer na mão
EMBEDDING_DIMENSION = 300  
MAX_TEXT_SIZE = max_text_length 
VOCAB_SIZE = text_vocab_size

In [107]:
def optimize (optimize, learning_rate):
  if optimize == 'Nadam':
    return Nadam(learning_rate=learning_rate, name="Nadam")
  
  if optimize == 'RMSProp':
    return RMSprop(learning_rate=learning_rate, name="RMSprop")
  
  if optimize == 'SGD':
    return SGD(learning_rate=learning_rate, name="SGD")

In [108]:
def create_cnn (vocab_size, embedding_dimen, max_text_size, dropout_rate ):

  modelo = keras.Sequential([
    keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION, input_length=max_text_size),
    # keras.layers.Embedding(input_dim=MATRIZ_CBOW_300_BASE_2.shape[0],
                            # output_dim=MATRIZ_CBOW_300_BASE_2.shape[1],
                            # weights=[MATRIZ_CBOW_300_BASE_2],
                            # embeddings_initializer=keras.initializers.Constant(MATRIZ_CBOW_300_BASE_2),
                            # input_length=max_text_size,
                            # trainable=True),                         
    keras.layers.Conv1D(max_text_size, 3, activation="relu"),
    keras.layers.Flatten(input_shape=(max_text_size,)),
    keras.layers.Dense(max_text_size, activation='relu'),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1, activation='sigmoid')
    ])
  
  return modelo 

In [109]:
def train_model(input_data, output_label, embedding_dimen, batch_size, epochs, validation_and_test_size, learning_rate, optimizer, dropout_rate):
    
    # Comentamos a divisão dos dados aqui pq fizemos a separação de maneira manual
    #x_main, x_test, y_main, y_test = train_test_split(input_data, output_label, test_size=0.2, stratify=output_label, random_state=42)
    
    #x_train, x_val, y_train, y_val = train_test_split(x_main, y_main, test_size=0.2, stratify=y_main, random_state=42)

    opt = optimize(optimizer, learning_rate)
    
    model = create_cnn(VOCAB_SIZE, EMBEDDING_DIMENSION, MAX_TEXT_SIZE, dropout_rate)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy'])        
    
    ra_val = RocAucEvaluation(validation_data=(x_val, y_val), interval = 1)
    
    csv_logger = CSVLogger('log.csv', append=False, separator=';')                                                                                       
    
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), verbose=0, use_multiprocessing=True,  callbacks = [ra_val, csv_logger])
  
    scores = model.evaluate(x_test, y_test, verbose=0)
    
    return history, model, x_test, y_test, ra_val, scores, csv_logger


In [110]:
# history, model1, x_test, y_test, ra_val, scores, csv_logger = train_model(input_data, output_label, EMBEDDING_DIMENSION, 32, 5, 0.2, 0.001, 'Nadam', 0.2)

In [111]:
# plot dos gráficos
count = 1
for BS in BATCH_SIZE:
  for LR in LEARN_RATE:
    for OT in OTMIZADORES:
      for DO in DROPOUT:
        print("Iteração "+str(count))
        count+=1
        
        history, model1, x_test, y_test, ra_val, scores, csv_logger = train_model(input_data, output_label, EMBEDDING_DIMENSION, BS, EPOCHS, VAL_AND_TST_SIZE, LR, OT, DO)

        #
        y_predito = model1.predict(x_test).round()
        y_gabarito = y_test
        
        # Parâmetros
        # rede base otimizador acuracia val_loss learning_rate dropout batch_size roc_curve train_resume confusion_matrix
        NOME_REDE = REDE
        NOME_BASE = BASE_0
        NOME_OTIMIZADOR = OT
        ACURACIA = scores[1]
        VAL_LOSS = scores[0] 
        LEARNING_RATE = LR
        DROPOUT_RATE = DO
        BATCH = BS
        ROC_CURVE = json.dumps({
            "false_positive_rate": list(ra_val.false_positive_rate),
            "true_positive_rate": list(ra_val.true_positive_rate),
            "score": ra_val.score})
        TRAIN_RESUME = json.dumps(pd.read_csv('log.csv',sep=';').to_dict()) 
        
        confusion = confusion_matrix(y_gabarito, y_predito)
        CONFUSION_MATRIX = json.dumps({
            "00": int(confusion[0][0]),
            "01": int(confusion[0][1]),
            "10": int(confusion[1][0]),
            "11": int(confusion[1][1])
        })
 
        # Salvando resultado do modelo
        registro_resultado = {resultados.columns[0]: NOME_REDE, 
                              resultados.columns[1]: NOME_BASE, 
                              resultados.columns[2]: NOME_OTIMIZADOR,
                              resultados.columns[3]: ACURACIA, 
                              resultados.columns[4]: VAL_LOSS,
                              resultados.columns[5]: LEARNING_RATE,
                              resultados.columns[6]: DROPOUT_RATE,
                              resultados.columns[7]: BATCH,
                              resultados.columns[8]: ROC_CURVE,
                              resultados.columns[9]: TRAIN_RESUME,
                              resultados.columns[10]: CONFUSION_MATRIX
                            }

        resultados = resultados.append(registro_resultado, ignore_index=True)

Iteração 1
Iteração 2
Iteração 3
Iteração 4
Iteração 5
Iteração 6
Iteração 7
Iteração 8
Iteração 9
Iteração 10
Iteração 11
Iteração 12
Iteração 13
Iteração 14
Iteração 15
Iteração 16
Iteração 17
Iteração 18
Iteração 19
Iteração 20
Iteração 21
Iteração 22
Iteração 23
Iteração 24
Iteração 25
Iteração 26
Iteração 27
Iteração 28
Iteração 29
Iteração 30
Iteração 31
Iteração 32
Iteração 33
Iteração 34
Iteração 35
Iteração 36


In [112]:
resultados

,rede,base,otimizador,acuracia,val_loss,learning_rate,dropout,batch_size,roc_curve,train_resume,confusion_matrix
0,CNN,BASE 2,Nadam,0.612474,1.575691,0.0001,0.1,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 235, ""01"": 254, ""10"": 125, ""11"": 364}"
1,CNN,BASE 2,Nadam,0.619632,1.603469,0.0001,0.2,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 251, ""01"": 238, ""10"": 134, ""11"": 355}"
2,CNN,BASE 2,Nadam,0.615542,1.554839,0.0001,0.3,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 229, ""01"": 260, ""10"": 116, ""11"": 373}"
3,CNN,BASE 2,RMSProp,0.601227,1.779262,0.0001,0.1,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 210, ""01"": 279, ""10"": 111, ""11"": 378}"
4,CNN,BASE 2,RMSProp,0.609407,1.659840,0.0001,0.2,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 235, ""01"": 254, ""10"": 128, ""11"": 361}"
...,...,...,...,...,...,...,...,...,...,...,...
67,CNN,BASE 0,RMSProp,0.619632,8.057823,0.0010,0.2,64,"{""false_positive_rate"": [0.0, 0.18813905930470...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 272, ""01"": 217, ""10"": 155, ""11"": 334}"
68,CNN,BASE 0,RMSProp,0.604294,7.291866,0.0010,0.3,64,"{""false_positive_rate"": [0.0, 0.23108384458077...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 220, ""01"": 269, ""10"": 118, ""11"": 371}"
69,CNN,BASE 0,SGD,0.502045,0.692827,0.0010,0.1,64,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0020...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 23, ""01"": 466, ""10"": 21, ""11"": 468}"
70,CNN,BASE 0,SGD,0.517382,0.693209,0.0010,0.2,64,"{""false_positive_rate"": [0.0, 0.00204498977505...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 378, ""01"": 111, ""10"": 361, ""11"": 128}"


In [113]:
# Caso queira deletar uma linha use o código abaixo
# Use a propriedade label para especificar o índice da linha
#resultados = resultados.drop(labels=1, axis=0)
#resultados = resultados.reset_index(drop=True)
#resultados

In [114]:
# Salva arquivo de saída
resultados.to_csv(PATH_ARQ_SAIDA)

In [115]:
# Fazer a separação de Treinamento, Teste e validação 
  # Alterar todos os modelos para ler desse novo jeito
# Ver NaiveBayes, colocar na tabela
  # Falta salvar o treinamento no arquivo de resultados
# Ver ShallowNet, colocar na tabela
  # ler igual a tarefa1
# Rever LSTM
  # rever o foward e backward layer
# Prioridade é fechar o Resultado ( Tabelas )
  # Rodar tudo
# Ajustar Tabelas ( embelezar )
# Ajustar Graficos ( embelezar )
# Ver BiLstm / Otmizador 
# Gerar e Salvar os Graficos 
# Escrever Resultados
# Revisar Monografia
# Preparar apresentação

grafico Acuracy b1  
		b1 cnn melhor
		b1 lstm melhor
		.
		.

grafico Loss b1  
		b1 cnn melhor
		b1 lstm melhor
		.
		.

grafico Acuracy b2 
		b2 cnn melhor
		b2 lstm melhor
		.
		.

grafico Loss b2 
		b2 cnn melhor
		b2 lstm melhor
		.
		.

grafico Acuracy b0
		bo cnn melhor
		b0 lstm melhor

grafico Loss b0
		b0 cnn melhor
		b0 lstm melhor
		.
		.
